# ID3 Algorithm

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# prepare data
outlook = 'overcast,overcast,overcast,overcast,rainy,rainy,rainy,rainy,rainy,sunny,sunny,sunny,sunny,sunny'.split(',')
temp = 'hot,cool,mild,hot,mild,cool,cool,mild,mild,hot,hot,mild,cool,mild'.split(',')
humidity = 'high,normal,high,normal,high,normal,normal,normal,high,high,high,high,normal,normal'.split(',')
windy = 'FALSE,TRUE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,TRUE'.split(',')
play = 'yes,yes,yes,yes,yes,yes,no,yes,no,no,no,no,yes,yes'.split(',')

In [5]:
print(outlook)
print(play)
print(len(outlook))
print(len(play))

['overcast', 'overcast', 'overcast', 'overcast', 'rainy', 'rainy', 'rainy', 'rainy', 'rainy', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny']
['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'yes']
14
14


In [6]:
dataset ={'outlook':outlook,'temp':temp,'humidity':humidity,'windy':windy,'play':play}
df = pd.DataFrame(dataset,columns=['outlook','temp','humidity','windy','play'])

In [7]:
df

,outlook,temp,humidity,windy,play
0,overcast,hot,high,FALSE,yes
1,overcast,cool,normal,TRUE,yes
2,overcast,mild,high,TRUE,yes
3,overcast,hot,normal,FALSE,yes
4,rainy,mild,high,FALSE,yes
5,rainy,cool,normal,FALSE,yes
6,rainy,cool,normal,TRUE,no
7,rainy,mild,normal,FALSE,yes
8,rainy,mild,high,TRUE,no
9,sunny,hot,high,FALSE,no


In [8]:
print(len(df)) #finding no of rows
print(len(df['play']))
print(df.columns) #finding no of columns
print(df['play'].value_counts())
print(df['temp'][df['temp']=='hot'][df['play']=='yes']) # to select multi features simultaneosly
print(df[df['temp']=='hot'])
clValue,counts = np.unique(['play','gym','play'],return_counts=True)
print(clValue,counts)

14
14
Index(['outlook', 'temp', 'humidity', 'windy', 'play'], dtype='object')
yes    9
no     5
Name: play, dtype: int64
0    hot
3    hot
Name: temp, dtype: object
     outlook temp humidity  windy play
0   overcast  hot     high  FALSE  yes
3   overcast  hot   normal  FALSE  yes
9      sunny  hot     high  FALSE   no
10     sunny  hot     high   TRUE   no
['gym' 'play'] [1 2]


In [9]:
def find_entropy(df):
    Class = df.keys()[-1] #play
    entropy = 0
    vals = df[Class].unique() #yes or no
    for value in vals:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy

In [10]:
print(find_entropy(df))

0.9402859586706311


In [11]:
def find_entropy_attribute(df,attribute):
    Class = df.keys()[-1]
    target_variables = df[Class].unique()
    variables = df[attribute].unique() #features like hot, cool, mild for temp
    entropy1 = 0
    for var in variables:
        entropy = 0
        for tar_var in target_variables:
            num = len(df[attribute][df[attribute]==var][df[Class]==tar_var])
            den = len(df[attribute][df[attribute]==var])
            fraction = num/(den)
            if fraction==0:
                continue
            else:
                entropy += -fraction*np.log2(fraction)
        # calculate avg-Info    
        fraction2 = den/len(df)
        entropy1 += -fraction2*entropy
    return abs(entropy1)

In [12]:
#check for one attribute entropy
print(find_entropy_attribute(df,'outlook'))

0.6935361388961918


In [13]:
list_entropy_attributes = [find_entropy_attribute(df,i) for i in df.columns[0:-1]] #entropy of all attributes except last one
print(list_entropy_attributes)

[0.6935361388961918, 0.9110633930116763, 0.7884504573082896, 0.8921589282623617]


In [19]:
def find_winner(df):
    ig = []
    for key in df.keys()[:-1]:
        ig.append(find_entropy(df)-find_entropy_attribute(df,key))
    #print(ig)
    return df.keys()[:-1][np.argmax(ig)]

In [15]:
print(find_winner(df))

[0.24674981977443933, 0.02922256565895487, 0.15183550136234159, 0.04812703040826949]
outlook


In [16]:
def get_subtable(df,node,value):
    return df[df[node]==value].reset_index(drop=True)

In [20]:
def build_tree(df,tree=None):
    Class = df.keys()[-1]
    node = find_winner(df)
    
    att_val = np.unique(df[node]) # find unique values of root node
    if tree is None:
        tree = {}
        tree[node] = {}
        for val in att_val:
            subtable = get_subtable(df,node,val)
            clValue,counts = np.unique(subtable['play'],return_counts=True)
            if len(counts)==1:
                tree[node][val] = clValue[0]
            else:
                tree[node][val] = build_tree(subtable)
    return tree

In [21]:
t = build_tree(df)
#print(t)
import pprint
pprint.pprint(t)

{'outlook': {'overcast': 'yes',
             'rainy': {'windy': {'FALSE': 'yes', 'TRUE': 'no'}},
             'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}
